<a href="https://colab.research.google.com/github/Lord0fTheFlies/NLP-Neuromatch-Project/blob/main/NMA_Number_of_Rows_per_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: NOTE: If your import is failing due to a missing package, you can
# manually install dependencies using either !pip or !apt.
# The missing package is "datasets"

!pip install datasets
!pip install wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import wandb
import numpy as np
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding

In [ ]:
ds = load_dataset("dhivyeshrk/Disease-Symptom-Extensive-Clean")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/246945 [00:00<?, ? examples/s]

In [ ]:
# Step 1: Read the CSV file into a DataFrame
df = pd.read_csv("hf://datasets/dhivyeshrk/Disease-Symptom-Extensive-Clean/Final_Augmented_dataset_Diseases_and_Symptoms.csv")


labels_list = []

# Iterate over each row in the aggregated DataFrame
for index, row in df.iterrows():
    disease_name = row[df.columns[0]]  # Get the disease name
    labels = []  # Temporary list to store labels for the current disease

    # Iterate over the symptom columns (starting from index 1)
    for col in df.columns[1:]:
        if row[col] == 1:
            labels.append(col)  # Append the column label if value is 1

    # Concatenate the labels into a string separated by commas
    labels_string = ", ".join(labels)
    labels_list.append(labels_string)  # Append the concatenated string to the list

# Add the concatenated labels as a new column in the aggregated DataFrame
df['Symptom_labels'] = labels_list

# Initialize a list of columns to keep
columns_to_keep = ['diseases', 'Symptom_labels']

# Drop all columns except 'disease' and 'Symptom_labels'
LLM_df = df[columns_to_keep]

LLM_df

,diseases,Symptom_labels
0,panic disorder,"anxiety and nervousness, shortness of breath, ..."
1,panic disorder,"shortness of breath, depressive or psychotic s..."
2,panic disorder,"anxiety and nervousness, depression, shortness..."
3,panic disorder,"anxiety and nervousness, depressive or psychot..."
4,panic disorder,"anxiety and nervousness, depression, insomnia,..."
...,...,...
246940,open wound of the nose,"diminished hearing, facial pain, nosebleed"
246941,open wound of the nose,"diminished hearing, nosebleed"
246942,open wound of the nose,"diminished hearing, nosebleed"
246943,open wound of the nose,"diminished hearing, headache, nosebleed"


**Analyzing consistency across entries per disease**

In [ ]:
import pandas as pd

def analyze_disease_rows(df):
    # Group by the disease name and count the number of rows per disease
    disease_counts = df.groupby('diseases').size()

    # Calculate the maximum, minimum, average, and variation in counts
    max_count = disease_counts.max()
    min_count = disease_counts.min()
    avg_count = disease_counts.mean()
    variation = max_count - min_count

    # Identify diseases with the maximum and minimum number of rows
    max_disease = disease_counts.idxmax()
    min_disease = disease_counts.idxmin()

    # Display the results
    print(f"Maximum number of rows for a disease: {max_count} (Disease: {max_disease})")
    print(f"Minimum number of rows for a disease: {min_count} (Disease: {min_disease})")
    print(f"Average number of rows per disease: {avg_count:.2f}")
    print(f"Variation in number of rows: {variation}")

    # Optionally, return the counts for further analysis
    return disease_counts

# Use the function with your DataFrame
disease_counts = analyze_disease_rows(df)


Maximum number of rows for a disease: 1219 (Disease: cystitis)
Minimum number of rows for a disease: 1 (Disease: chronic ulcer)
Average number of rows per disease: 319.46
Variation in number of rows: 1218


**Reducing the amount of rows**

*   Filtering out diseases with less than 50 entries
*   Limiting all diseases to only 50 entries



In [ ]:
import pandas as pd

def filter_and_limit_diseases(df, min_rows=50, limit_rows=50):
    # Group by the disease name and count the number of rows per disease
    disease_counts = df.groupby('diseases').size()

    # Identify diseases with at least min_rows entries
    diseases_to_keep = disease_counts[disease_counts >= min_rows].index.tolist()

    # Create a DataFrame with diseases that meet the min_rows criteria
    filtered_df = df[df['diseases'].isin(diseases_to_keep)]

    # Limit the number of rows for each disease to limit_rows
    limited_df = filtered_df.groupby('diseases').apply(lambda x: x.head(limit_rows)).reset_index(drop=True)

    # Identify diseases that have been filtered out
    diseases_filtered_out = disease_counts[disease_counts < min_rows].index.tolist()

    # Calculate the average row count per disease after limiting
    avg_row_count = limited_df.groupby('diseases').size().mean()

    # Display the results
    print(f"Diseases with at least {min_rows} entries: {len(diseases_to_keep)}")
    print(f"Diseases filtered out ({len(diseases_filtered_out)}): {diseases_filtered_out}")
    print(f"Average row count per disease after limiting: {avg_row_count:.2f}")

    return limited_df, diseases_filtered_out

# Use the function with your DataFrame
filtered_limited_df, diseases_filtered_out = filter_and_limit_diseases(df)

# Display the new DataFrame
filtered_limited_df



Diseases with at least 50 entries: 527
Diseases filtered out (246): ['abscess of the lung', 'acanthosis nigricans', 'acariasis', 'acute fatty liver of pregnancy (aflp)', 'adrenal adenoma', 'adrenal cancer', 'allergy to animals', 'amblyopia', 'amyloidosis', 'anal fistula', 'anemia due to malignancy', 'anemia of chronic disease', 'aphakia', 'aspergillosis', 'atrophy of the corpus cavernosum', 'avascular necrosis', 'birth trauma', 'blepharospasm', 'bone spur of the calcaneous', 'breast cancer', 'breast cyst', 'bunion', 'carcinoid syndrome', 'cat scratch disease', 'central atherosclerosis', 'cervical disorder', 'cholesteatoma', 'chronic inflammatory demyelinating polyneuropathy (cidp)', 'chronic kidney disease', 'chronic ulcer', 'colorectal cancer', 'congenital heart defect', 'congenital malformation syndrome', 'connective tissue disorder', 'conversion disorder', 'cryptococcosis', 'cryptorchidism', 'cushing syndrome', 'cyst of the eyelid', 'cystic fibrosis', 'cysticercosis', 'decubitus ulc

,diseases,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,...,problems with orgasm,nose deformity,lump over jaw,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness,Symptom_labels
0,abdominal aortic aneurysm,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"shortness of breath, palpitations, back pain"
1,abdominal aortic aneurysm,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"shortness of breath, palpitations, sharp abdom..."
2,abdominal aortic aneurysm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"palpitations, arm swelling, back pain, burning..."
3,abdominal aortic aneurysm,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"shortness of breath, palpitations, arm swellin..."
4,abdominal aortic aneurysm,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"shortness of breath, palpitations, sharp abdom..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26345,yeast infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"suprapubic pain, sharp abdominal pain, frequen..."
26346,yeast infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"suprapubic pain, sharp abdominal pain, frequen..."
26347,yeast infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"suprapubic pain, sharp abdominal pain, vaginal..."
26348,yeast infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"suprapubic pain, sharp abdominal pain, vaginal..."


**Split the data between training (80%), test (10%) and validation (10%)**

In [ ]:
# Ensure consistency by setting a random seed
random_seed = 42

# Initial split: 80% training, 20% (test + validation)
train_df, temp_df = train_test_split(filtered_limited_df, test_size=0.2, stratify=filtered_limited_df['diseases'], random_state=random_seed)

# Secondary split: 50% of the temp_df to validation and 50% to test (10% each of the original data)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['diseases'], random_state=random_seed)

# Print the sizes of the splits to verify
print(f"Training set size: {train_df.shape[0]}")
print(f"Validation set size: {validation_df.shape[0]}")
print(f"Test set size: {test_df.shape[0]}")

# Optional: save the splits to CSV files
train_df.to_csv('train_data.csv', index=False)
validation_df.to_csv('validation_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

Training set size: 21080
Validation set size: 2635
Test set size: 2635
